In [7]:
import json
from pathlib import Path

import pandas as pd

### Exp 2: number of gaussians

In [8]:
results_base_path = Path('output/WAT/realtime4DGS-exp2')

In [9]:
gaussian_counts_paths = sorted([path for path in results_base_path.iterdir() if path.is_dir()], key=lambda x: int(x.name.split('_')[-1]))
gaussian_counts_paths

[PosixPath('output/WAT/realtime4DGS-exp2/gaussians_500000'),
 PosixPath('output/WAT/realtime4DGS-exp2/gaussians_1000000'),
 PosixPath('output/WAT/realtime4DGS-exp2/gaussians_2000000'),
 PosixPath('output/WAT/realtime4DGS-exp2/gaussians_3000000'),
 PosixPath('output/WAT/realtime4DGS-exp2/gaussians_4000000')]

In [10]:
# Helper functions
def get_nvs_metrics(scene_path: Path) -> dict:
    json_path = scene_path / 'results.json'
    if not json_path.exists():
        print(f"{json_path.name} for scene {json_path.parent.name} doesn't exist")
    with open(json_path, 'r') as f:
        data = json.load(f)
    data = data.get('ours_None', {})
    result = {
        'Gaussians': int(scene_path.parent.name.split('_')[-1]),
        'Scene': scene_path.name,
        'PSNR': data.get('PSNR', 'N/A'),
        'SSIM': data.get('SSIM', 'N/A'),
        'LPIPS (vgg)': data.get('LPIPS', 'N/A')
    }
    return result

def get_run_metrics(scene_path: Path) -> dict:
    wandb_run_path = scene_path / 'wandb' / 'latest-run' / 'files'
    metadata_json_path = wandb_run_path / 'wandb-metadata.json'
    summary_json_path = wandb_run_path / 'wandb-summary.json'

    if not metadata_json_path.exists() or not summary_json_path.exists():
        print(f"Wandb json paths can't be found for {scene_path.name}")
    
    with open(metadata_json_path, 'r') as f:
        data = json.load(f)
        gpu_model = data.get('gpu_nvidia')[0]['name']

    with open(summary_json_path, 'r') as f:
        data = json.load(f)
        try:
            runtime = int(data.get('_runtime') / 60)
        except:
            runtime = 'N/A'

    result = {
        'runtime (min)': runtime,
        'GPU model': gpu_model,
     }
    return result

def get_storage(scene_path: Path) -> float:
    checkpoint = scene_path / 'chkpnt30000.pth'
    if not checkpoint.exists():
        print(f"{checkpoint.name} for scene {scene_path.name} doesn't exist")
    storage_mb = round(checkpoint.stat().st_size / 1024**3, 3)
    return storage_mb

In [11]:
metrics = []
for path in gaussian_counts_paths:
    for scene in sorted(path.iterdir()):
        scene_metrics = get_nvs_metrics(scene)
        scene_metrics.update(get_run_metrics(scene))
        scene_metrics.update({'Storage (GB)': get_storage(scene)})
        metrics.append(scene_metrics)

In [12]:
df = pd.DataFrame(metrics)
df

,Gaussians,Scene,PSNR,SSIM,LPIPS (vgg),runtime (min),GPU model,Storage (GB)
0,500000,breville,29.928093,0.918685,0.174810,233,NVIDIA RTX A5000,0.967
1,500000,car_resized,22.773722,0.634445,0.393967,184,NVIDIA RTX A5000,0.988
2,500000,community,24.457262,0.749699,0.364576,248,NVIDIA RTX A5000,0.979
3,500000,dyson,26.224741,0.921506,0.234921,201,NVIDIA RTX A5000,0.940
4,500000,grill_resized,25.095129,0.730192,0.371249,197,NVIDIA RTX A5000,0.917
5,500000,kitchen,28.813145,0.921577,0.150615,230,NVIDIA RTX A5000,0.926
6,500000,living_room,25.873131,0.887814,0.223185,209,NVIDIA RTX A5000,0.975
7,500000,mac,30.121283,0.935708,0.204154,169,NVIDIA RTX A5000,0.933
8,500000,ninja,28.626543,0.925797,0.220191,182,NVIDIA RTX A5000,0.921
9,500000,spa,28.097721,0.886805,0.256477,228,NVIDIA RTX A5000,0.991


In [14]:
df.to_csv('WAT_realtime4DGS-exp2.csv', index=False)

In [21]:
df.groupby('Gaussians')[['runtime (min)', 'Storage (GB)']].mean()

,runtime (min),Storage (GB)
Gaussians,,
500000,209.454545,0.957273
1000000,239.181818,1.866909
2000000,172.909091,3.734091
3000000,190.545455,5.464182
4000000,269.363636,6.814364
